In [1]:
import pandas as pd
import pickle

In [2]:
file_path = "/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/cubert_embeddings/df_fe_embeded_by_line_cubert.pkl"
with open(file_path, "rb") as fh:
  df = pickle.load(fh)

/var/folders/d8/cr798cb57kzd5kz_r8g8jm380000gn/T/ipykernel_51882/2128490750.py:3: DeprecationWarning: numpy.core.numeric is deprecated and has been renamed to numpy._core.numeric. The numpy._core namespace contains private NumPy internals and its use is discouraged, as NumPy internals can change without warning in any release. In practice, most real-world usage of numpy.core is to access functionality in the public NumPy API. If that is the case, use the public NumPy API. If not, you are using NumPy internals. If you would still like to access an internal attribute, use numpy._core.numeric._frombuffer.
  df = pickle.load(fh)


In [3]:
df.head()

,sample_id,severity,method,label,embeded_sequence_sum,embeded_sequence_avg
0,12150,critical,public SystemDiagnosticsDTO createSystemDi...,1,"[[-42.401798248291016, -26.870685577392578, -4...","[[-0.6730444431304932, -0.42651882767677307, -..."
1,10168,critical,protected void addRequiredAlertProperties(Se...,1,"[[4.771414279937744, -9.410942077636719, -2.36...","[[0.9542828798294067, -1.8821884393692017, -0...."
2,12783,critical,"@SuppressWarnings(""try"")\n private void...",1,"[[11.647194862365723, -80.076416015625, -124.1...","[[0.0946926400065422, -0.6510277986526489, -1...."
3,12754,critical,public void validateDepositDetailForUpdate...,1,"[[-6.452081680297852, -117.95765686035156, -12...","[[-0.05760787054896355, -1.0531933307647705, -..."
4,4277,critical,@Override\n public ListenableFuture<Voi...,1,"[[8.61170482635498, -30.42780113220215, -61.42...","[[0.11184032261371613, -0.3951662480831146, -0..."


In [4]:
embeded_sum = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_sum']]
embeded_avg = df.loc[:, ['sample_id', 'severity', 'label', 'embeded_sequence_avg']]
embeded_sum.head()

,sample_id,severity,label,embeded_sequence_sum
0,12150,critical,1,"[[-42.401798248291016, -26.870685577392578, -4..."
1,10168,critical,1,"[[4.771414279937744, -9.410942077636719, -2.36..."
2,12783,critical,1,"[[11.647194862365723, -80.076416015625, -124.1..."
3,12754,critical,1,"[[-6.452081680297852, -117.95765686035156, -12..."
4,4277,critical,1,"[[8.61170482635498, -30.42780113220215, -61.42..."


In [5]:
print(embeded_sum["embeded_sequence_sum"].head())
print(embeded_sum["embeded_sequence_sum"].apply(type).value_counts())

0    [[-42.401798248291016, -26.870685577392578, -4...
1    [[4.771414279937744, -9.410942077636719, -2.36...
2    [[11.647194862365723, -80.076416015625, -124.1...
3    [[-6.452081680297852, -117.95765686035156, -12...
4    [[8.61170482635498, -30.42780113220215, -61.42...
Name: embeded_sequence_sum, dtype: object
embeded_sequence_sum
<class 'list'>    3027
Name: count, dtype: int64


In [6]:
def process_embedding(x):
    if x is None:
        return []  
    elif isinstance(x, pd.Series):
        return " ".join(map(str, x))
    else:
        return str(x)  

embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(process_embedding)


In [7]:
print(embeded_sum["embeded_sequence_sum"].head())

0    [[-42.401798248291016, -26.870685577392578, -4...
1    [[4.771414279937744, -9.410942077636719, -2.36...
2    [[11.647194862365723, -80.076416015625, -124.1...
3    [[-6.452081680297852, -117.95765686035156, -12...
4    [[8.61170482635498, -30.42780113220215, -61.42...
Name: embeded_sequence_sum, dtype: object


In [8]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [9]:
import re

def parse_flat_embedding(embedding_str):
    # Ensure input is a string
    if not isinstance(embedding_str, str):
        return []
    try:
        # Extract numerical values from the string
        values = re.findall(r"[-+]?\d*\.\d+|\d+", embedding_str)
        return [float(v) for v in values]
    except Exception as e:
        print(f"Error parsing: {embedding_str} -> {e}")
        return []


In [10]:
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(parse_flat_embedding)

In [11]:
n = len(embeded_sum["embeded_sequence_sum"][0])
print(f"Embedding length: {n}")

embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")


Embedding length: 768
Most common sequence length: 768
Count of rows with this length: 2985


In [12]:
 # Function to handle the trimming or padding of the sequence
embeded_sum["embeded_sequence_sum"] = embeded_sum["embeded_sequence_sum"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_sum["embeded_sequence_sum"].head())

0    [-42.401798248291016, -26.870685577392578, -46...
1    [4.771414279937744, -9.410942077636719, -2.367...
2    [11.647194862365723, -80.076416015625, -124.18...
3    [-6.452081680297852, -117.95765686035156, -129...
4    [8.61170482635498, -30.42780113220215, -61.426...
Name: embeded_sequence_sum, dtype: object


In [13]:
embed_lengths = embeded_sum["embeded_sequence_sum"].apply(len)
print(embed_lengths.value_counts())


embeded_sequence_sum
768    3027
Name: count, dtype: int64


In [14]:
# Create column names for embeddings
columns = [f'emb_{i+1}' for i in range(n)]

# Create a DataFrame for embedding values
data = pd.DataFrame(embeded_sum["embeded_sequence_sum"].to_list(), columns=columns)

# Add metadata columns
data['label'] = embeded_sum['label']
data['sample_id'] = embeded_sum['sample_id']
data['severity'] = embeded_sum['severity']

# Display the result
print(data.head())


       emb_1       emb_2       emb_3      emb_4      emb_5      emb_6  \
0 -42.401798  -26.870686  -46.018105   8.521990  15.175569 -18.864988   
1   4.771414   -9.410942   -2.367512   0.598740  -5.552438  -2.216153   
2  11.647195  -80.076416 -124.189705  58.185944  48.472561  24.540005   
3  -6.452082 -117.957657 -129.250961  41.561977 -21.422041 -83.653915   
4   8.611705  -30.427801  -61.426537  33.076874  20.033140 -68.134056   

        emb_7       emb_8      emb_9     emb_10  ...    emb_762    emb_763  \
0   -5.143667 -103.999161  52.696270  34.906448  ...  14.058058 -35.712307   
1    2.808177   -3.373450   4.429100   3.801935  ...   0.464715   2.624750   
2   36.780075  -63.776188  82.424889  20.396940  ...  31.705832   2.266652   
3  104.651474 -106.414589  20.539492 -15.075366  ... -34.709187 -47.284946   
4   14.036429  -13.502237  31.836353  34.797176  ...  21.287672  24.752357   

     emb_764    emb_765    emb_766     emb_767    emb_768  label  sample_id  \
0  -3.190466 

In [15]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_sum.pkl'
pd.to_pickle(data, dataset_path)

In [16]:
print(embeded_avg["embeded_sequence_avg"].head())
print(embeded_avg["embeded_sequence_avg"].apply(type).value_counts())

0    [[-0.6730444431304932, -0.42651882767677307, -...
1    [[0.9542828798294067, -1.8821884393692017, -0....
2    [[0.0946926400065422, -0.6510277986526489, -1....
3    [[-0.05760787054896355, -1.0531933307647705, -...
4    [[0.11184032261371613, -0.3951662480831146, -0...
Name: embeded_sequence_avg, dtype: object
embeded_sequence_avg
<class 'list'>    3027
Name: count, dtype: int64


In [17]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: " ".join(str(i) for i in x) if isinstance(x, pd.Series) else str(x)
)

In [18]:
n = len(embeded_avg["embeded_sequence_avg"][0])
print(f"Embedding length: {n}")

Embedding length: 15974


In [19]:
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(parse_flat_embedding)

In [20]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
length_counts = embed_lengths.value_counts()
most_common_length = length_counts.idxmax()

n = most_common_length
print(f"Most common sequence length: {most_common_length}")
print(f"Count of rows with this length: {length_counts[most_common_length]}")

Most common sequence length: 768
Count of rows with this length: 2742


In [21]:
# Function to handle the trimming or padding of the sequence
embeded_avg["embeded_sequence_avg"] = embeded_avg["embeded_sequence_avg"].apply(
    lambda x: (x[:n] if len(x) > n else x + [0] * (n - len(x))) if isinstance(x, list) else [0] * n
)

print(embeded_avg["embeded_sequence_avg"].head())

0    [-0.6730444431304932, -0.42651882767677307, -0...
1    [0.9542828798294067, -1.8821884393692017, -0.4...
2    [0.0946926400065422, -0.6510277986526489, -1.0...
3    [-0.05760787054896355, -1.0531933307647705, -1...
4    [0.11184032261371613, -0.3951662480831146, -0....
Name: embeded_sequence_avg, dtype: object


In [22]:
embed_lengths = embeded_avg["embeded_sequence_avg"].apply(len)
print(embed_lengths.value_counts())

embeded_sequence_avg
768    3027
Name: count, dtype: int64


In [23]:
columns = [f'emb_{i+1}' for i in range(n)]
data = pd.DataFrame(embeded_avg["embeded_sequence_avg"].to_list(), columns=columns)
data['label'] = embeded_avg['label']
data['sample_id'] = embeded_avg['sample_id']
data['severity'] = embeded_avg['severity']
data.head()

,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,emb_9,emb_10,...,emb_762,emb_763,emb_764,emb_765,emb_766,emb_767,emb_768,label,sample_id,severity
0,-0.673044,-0.426519,-0.730446,0.135270,0.240882,-0.299444,-0.081646,-1.650780,0.836449,0.554071,...,0.223144,-0.566862,-0.050642,0.790394,-0.679357,0.765604,-0.684285,1,12150,critical
1,0.954283,-1.882188,-0.473502,0.119748,-1.110488,-0.443231,0.561635,-0.674690,0.885820,0.760387,...,0.092943,0.524950,-0.121175,0.480204,-0.743649,1.266763,-0.823656,1,10168,critical
2,0.094693,-0.651028,-1.009672,0.473056,0.394086,0.199512,0.299025,-0.518506,0.670121,0.165829,...,0.685719,0.257771,0.018428,0.311697,0.502811,-0.001555,1.064599,1,12783,critical
3,-0.057608,-1.053193,-1.154026,0.371089,-0.191268,-0.746910,0.934388,-0.950130,0.183388,-0.134601,...,-0.309903,-0.422187,-0.038162,0.447833,-0.307583,0.608918,-0.078509,1,12754,critical
4,0.111840,-0.395166,-0.797747,0.429570,0.260171,-0.884858,0.182291,-0.175354,0.413459,0.451911,...,0.276463,0.321459,0.029099,0.301428,-0.008018,0.659282,-0.336579,1,4277,critical


In [24]:
dataset_path = '/Users/mac/Desktop/Code_Smell_Detection/dataset/feature_envy/cubert_embeddings/processed_embeddings/cubert_embed_line_by_line_avg.pkl'
pd.to_pickle(data, dataset_path)